In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import to_categorical
%matplotlib inline

Using TensorFlow backend.
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user10\Anaconda3\envs\P

In [2]:
df = pd.read_csv('./data/3227set.csv', index_col = 0)

In [3]:
df['mid1'] = (df['Ask1'] + df['Bid1']) / 2

In [4]:
df = df.reset_index(drop = True)

In [5]:
df = df.drop([0, 253059])
df = df.reset_index(drop = True)

In [6]:
df = df.drop(['next_mid1'], axis = 1)

In [7]:
sum(df['Ask2'] == 0)

44

In [8]:
sum(df['Ask3'] == 0)

510

In [9]:
sum(df['Ask4'] == 0)

1273

In [10]:
sum(df['Ask5'] == 0)

1617

In [11]:
df

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
1,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
3,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
4,92,126.5,127.0,8,78,126.0,127.5,41,59,125.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253053,212,130.0,130.5,23,142,129.5,131.0,24,316,129.0,...,-2.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,-1.0,130.25
253054,212,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
253055,213,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
253056,213,130.0,130.5,15,142,129.5,131.0,25,312,129.0,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [12]:
#normalize
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm
#rolling windows
def train_windows(df, ref_day=5, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["mid1"]))
    return np.array(X_train), np.array(Y_train)
#LSTM
def lstm_stock_model(shape):
    model = Sequential()
    model.add(LSTM(256, input_shape=(shape[1], shape[2]), return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.add(Flatten())
    model.add(Dense(5,activation='linear'))
    model.add(Dense(1,activation='linear'))
    model.compile(loss="mean_absolute_error", optimizer="adam",metrics=['mean_absolute_error'])
    model.summary()
    return model


In [19]:
df

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
1,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
3,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
4,92,126.5,127.0,8,78,126.0,127.5,41,59,125.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253053,212,130.0,130.5,23,142,129.5,131.0,24,316,129.0,...,-2.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,-1.0,130.25
253054,212,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
253055,213,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
253056,213,130.0,130.5,15,142,129.5,131.0,25,312,129.0,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [20]:
df1 = df[:10000]
df1

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
1,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
3,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
4,92,126.5,127.0,8,78,126.0,127.5,41,59,125.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,42,122.0,122.5,32,79,121.5,123.0,178,127,121.0,...,0.0,0.0,0.0,0.0,-10.0,-3.0,0.0,0.0,0.0,122.25
9996,42,122.0,122.5,32,79,121.5,123.0,177,132,121.0,...,-1.0,0.0,0.0,-10.0,0.0,0.0,5.0,0.0,0.0,122.25
9997,39,122.0,122.5,32,79,121.5,123.0,177,132,121.0,...,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,122.25
9998,36,122.0,122.5,32,79,121.5,123.0,179,133,121.0,...,2.0,0.0,0.0,0.0,-3.0,0.0,1.0,0.0,0.0,122.25


In [13]:
df.shape[0]*0.3

75917.4

In [21]:
test = df1[-round(df1.shape[0]*0.3):]
test

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
7000,14,122.0,122.5,85,139,121.5,123.0,86,252,121.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,122.25
7001,13,122.0,122.5,85,139,121.5,123.0,86,252,121.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,122.25
7002,13,122.0,122.5,84,139,121.5,123.0,87,252,121.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.25
7003,21,122.0,122.5,84,139,121.5,123.0,86,252,121.0,...,-1.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,122.25
7004,19,122.0,122.5,84,139,121.5,123.0,86,252,121.0,...,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,122.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,42,122.0,122.5,32,79,121.5,123.0,178,127,121.0,...,0.0,0.0,0.0,0.0,-10.0,-3.0,0.0,0.0,0.0,122.25
9996,42,122.0,122.5,32,79,121.5,123.0,177,132,121.0,...,-1.0,0.0,0.0,-10.0,0.0,0.0,5.0,0.0,0.0,122.25
9997,39,122.0,122.5,32,79,121.5,123.0,177,132,121.0,...,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,122.25
9998,36,122.0,122.5,32,79,121.5,123.0,179,133,121.0,...,2.0,0.0,0.0,0.0,-3.0,0.0,1.0,0.0,0.0,122.25


In [22]:
test = test.reset_index()
test

,index,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,7000,14,122.0,122.5,85,139,121.5,123.0,86,252,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,122.25
1,7001,13,122.0,122.5,85,139,121.5,123.0,86,252,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,122.25
2,7002,13,122.0,122.5,84,139,121.5,123.0,87,252,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.25
3,7003,21,122.0,122.5,84,139,121.5,123.0,86,252,...,-1.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,122.25
4,7004,19,122.0,122.5,84,139,121.5,123.0,86,252,...,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,122.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,9995,42,122.0,122.5,32,79,121.5,123.0,178,127,...,0.0,0.0,0.0,0.0,-10.0,-3.0,0.0,0.0,0.0,122.25
2996,9996,42,122.0,122.5,32,79,121.5,123.0,177,132,...,-1.0,0.0,0.0,-10.0,0.0,0.0,5.0,0.0,0.0,122.25
2997,9997,39,122.0,122.5,32,79,121.5,123.0,177,132,...,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,122.25
2998,9998,36,122.0,122.5,32,79,121.5,123.0,179,133,...,2.0,0.0,0.0,0.0,-3.0,0.0,1.0,0.0,0.0,122.25


In [23]:
train = df1[:round(df1.shape[0]*0.7)]
train

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1
0,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
1,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
3,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
4,92,126.5,127.0,8,78,126.0,127.5,41,59,125.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,8,122.0,122.5,85,139,121.5,123.0,86,251,121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.25
6996,9,122.0,122.5,85,139,121.5,123.0,86,251,121.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,122.25
6997,14,122.0,122.5,85,139,121.5,123.0,86,251,121.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,122.25
6998,15,122.0,122.5,85,139,121.5,123.0,86,251,121.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,122.25


In [24]:
#720's 5sec -> 1hours
X_train, Y_train = train_windows(train, 10, 1)
X_test, Y_test = train_windows(test, 10, 1)

In [25]:
model = lstm_stock_model(X_train.shape)
callback = EarlyStopping(monitor="mean_absolute_error", patience=10, verbose=1, mode="auto")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 256)           329728    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             257       
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 855,358
Trainable params: 855,358
Non-trainable params: 0
________________________________________________

In [ ]:
history = model.fit(X_train, Y_train, epochs=1000, batch_size=5, validation_split=0.1, callbacks=[callback],shuffle=True)


Train on 6290 samples, validate on 699 samples
Epoch 1/1000
3160/6290 [==============>...............] - ETA: 31s - loss: 11.7780 - mean_absolute_error: 11.7780